# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/ex13/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [9]:
#pd.DataFrame(ext_data['train']['features'])

In [10]:
#from sklearn.preprocessing import StandardScaler
#from tqdm import tqdm

In [11]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [12]:
#pd.DataFrame(ext_data['train']['features'])

In [13]:
#hoge1 = scaler.fit_transform(ext_data['train']['features'])
#hoge2 = scaler.transform(ext_data['valid_source']['features'])
#hoge3 = scaler.transform(ext_data['valid_target']['features'])

In [14]:
#print(hoge1.mean(axis=0))
#print(hoge2.mean(axis=0))
#print(hoge3.mean(axis=0))

In [15]:
#scaler.transform(ext_data['valid_source']['features'])

In [16]:
#scaler = StandardScaler()
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
#scalers = []
#for phase in ['train', 'valid_source', 'valid_target']:
#    section_types = com.get_section_types(ext_data[phase]['wav_names'])
#    features = ext_data[phase]['features']
#    for section in tqdm(np.unique(section_types)):
#        if phase == 'train':
#            idx = np.where(section_types == section)[0]
#            sample = ext_data[phase]['features'][idx, :]
#            sample = scaler.fit_transform(sample)
#            ext_data[phase]['features'][idx, :] = sample
#            scalers.append(scaler)
#        else:
#            idx = np.where(section_types == section)[0]
#            sample = ext_data[phase]['features'][idx, :]
#            sample = scalers[section].transform(sample)
#            ext_data[phase]['features'][idx, :] = sample

In [17]:
#import seaborn as sns

In [18]:
#print(hoge1.std(axis=0).mean())
#print(hoge2.std(axis=0).mean())
#print(hoge3.std(axis=0).mean())
#sns.distplot(hoge1.std(axis=0))
#plt.show()
#sns.distplot(hoge2.std(axis=0))
#plt.show()
#sns.distplot(hoge3.std(axis=0))
#plt.show()

In [19]:
# print(ext_data['train']['features'].std(axis=0).mean())
# print(ext_data['valid_source']['features'].std(axis=0).mean())
# print(ext_data['valid_target']['features'].std(axis=0).mean())

# sns.distplot(ext_data['train']['features'].std(axis=0))
# plt.show()
# sns.distplot(ext_data['valid_source']['features'].std(axis=0))
# plt.show()
# sns.distplot(ext_data['valid_target']['features'].std(axis=0))
# plt.show()

In [20]:
# plt.plot(ext_data['valid_target']['features'].mean(axis=0))

## training

In [21]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [22]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [23]:
#for machine_type in machine_types:
#    run(machine_type)

In [24]:
run(machine_types[2])

2021-05-29 16:25:43,316 - 00_train.py - INFO - TARGET MACHINE_TYPE: valve
2021-05-29 16:25:43,316 - 00_train.py - INFO - MAKE DATA_LOADER
2021-05-29 16:25:43,376 - 00_train.py - INFO - TRAINING
2021-05-29 16:25:46,391 - pytorch_modeler.py - INFO - train
  0%|          | 0/12 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 12/12 [00:00<00:00, 15.07it/s]
2021-05-29 16:25:47,191 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 86.47it/s]
2021-05-29 16:25:47,234 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 105.30it/s]


,AUC,pAUC
Source_0,0.501700,0.517368
Source_1,0.519500,0.507368
Source_2,0.425400,0.505789
Target_0,0.491700,0.531579
Target_1,0.592700,0.552632
Target_2,0.460300,0.525789
mean,0.498550,0.523421
h_mean,0.493327,0.522943


2021-05-29 16:25:47,751 - pytorch_modeler.py - INFO - epoch:1/300, train_losses:60329.814779, val_AUC_hmean:0.493327, val_pAUC_hmean:0.522943, best_flag:True
2021-05-29 16:25:47,752 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 35.26it/s]
2021-05-29 16:25:48,095 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 93.59it/s]
2021-05-29 16:25:48,132 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 95.30it/s]
2021-05-29 16:25:48,178 - pytorch_modeler.py - INFO - epoch:2/300, train_losses:47265.572266, val_AUC_hmean:0.522320, val_pAUC_hmean:0.519371, best_flag:False
2021-05-29 16:25:48,179 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 35.20it/s]
2021-05-29 16:25:48,522 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 95.00it/s]
2021-05-29 16:25:48,562 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 102.81it/s]
2021-05-29 16:

...

2021-05-29 16:27:43,221 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 38.11it/s]
2021-05-29 16:27:43,537 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 112.03it/s]
2021-05-29 16:27:43,571 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 121.77it/s]
2021-05-29 16:27:43,607 - pytorch_modeler.py - INFO - epoch:263/300, train_losses:242.776990, val_AUC_hmean:0.653045, val_pAUC_hmean:0.582124, best_flag:False
2021-05-29 16:27:43,609 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 38.90it/s]
2021-05-29 16:27:43,921 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 109.56it/s]
2021-05-29 16:27:43,955 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 110.60it/s]
2021-05-29 16:27:43,993 - pytorch_modeler.py - INFO - epoch:264/300, train_losses:260.208145, val_AUC_hmean:0.604812, val_pAUC_hmean:0.531406, best_flag:False
2021-05-29

100%|██████████| 2/2 [00:00<00:00, 108.15it/s]
2021-05-29 16:27:49,806 - pytorch_modeler.py - INFO - epoch:279/300, train_losses:161.775887, val_AUC_hmean:0.595665, val_pAUC_hmean:0.550956, best_flag:False
2021-05-29 16:27:49,807 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 39.24it/s]
2021-05-29 16:27:50,115 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 103.11it/s]
2021-05-29 16:27:50,154 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 99.29it/s]
2021-05-29 16:27:50,196 - pytorch_modeler.py - INFO - epoch:280/300, train_losses:160.333317, val_AUC_hmean:0.563905, val_pAUC_hmean:0.548669, best_flag:False
2021-05-29 16:27:50,198 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 36.81it/s]
2021-05-29 16:27:50,526 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 103.92it/s]
2021-05-29 16:27:50,562 - pytorch_modeler.py - INFO - valid_target
100%|██████

100%|██████████| 12/12 [00:00<00:00, 40.34it/s]
2021-05-29 16:27:56,326 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 110.15it/s]
2021-05-29 16:27:56,362 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 120.63it/s]
2021-05-29 16:27:56,398 - pytorch_modeler.py - INFO - epoch:296/300, train_losses:153.247430, val_AUC_hmean:0.544637, val_pAUC_hmean:0.549743, best_flag:False
2021-05-29 16:27:56,399 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 38.59it/s]
2021-05-29 16:27:56,712 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 112.40it/s]
2021-05-29 16:27:56,746 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 104.42it/s]
2021-05-29 16:27:56,785 - pytorch_modeler.py - INFO - epoch:297/300, train_losses:156.404871, val_AUC_hmean:0.495017, val_pAUC_hmean:0.521623, best_flag:False
2021-05-29 16:27:56,786 - pytorch_modeler.py - INFO - train
100%|█████

,AUC,pAUC
Source_0,0.747400,0.678421
Source_1,0.545400,0.523158
Source_2,0.632400,0.616316
Target_0,0.765400,0.621053
Target_1,0.644300,0.589474
Target_2,0.538400,0.511579
mean,0.645550,0.590000
h_mean,0.633667,0.584263


elapsed time: 134.625766039 [sec]
